## Transform Orders Data - String to Json
<ol>
<li>Pre-process the json string to fix the data quality issues</li>
<li>Transform Json String to Json Object</li>
<li>Write transformed data to silver schema</li>
</ol>

In [0]:
Select * from gizmobox_cat.bronze.v_order;

### 1.Pre-process the json string to fix the data quality issues

In [0]:
CREATE OR REPLACE TEMPORARY VIEW tv_order_fixed_int AS
Select value, 
       regexp_replace(regexp_replace(regexp_replace(value, '([0-9]{4}-[0-9]{2}-[0-9]{2})', 
       '"\$1"'),'""','"'),
       '" ([0-9]{2}:[0-9]{2}:[0-9]{2})"', 
       ' \$1"') AS fixed_value
from gizmobox_cat.bronze.v_order;

In [0]:
--  CREATE OR REPLACE TEMPORARY VIEW Tv_order_fixed AS
-- Select value,
--        regexp_replace(value, '"order_date":(\\d{4}-\\d{2}-\\d{2})', '"order_date":"\$1"') as fixed_value
-- from gizmobox_cat.bronze.v_order;

###2.Transform Json String to Json Object
<ul>
<li><a href ='https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/schema_of_json'>schema_of_json function</a></li>
<li><a href='https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/from_json'>from_json function</a></li>
</ul>

In [0]:
select schema_of_json(fixed_value),value from tv_order_fixed_int LIMIT 1;
-- use to create schema from string formatted json which we can parse on from_json function to create json object

In [0]:
select from_json(fixed_value,'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') AS json_value,
fixed_value from tv_order_fixed_int ;


###3.Write transformed data to silver schema

In [0]:
CREATE TABLE IF NOT EXISTS gizmobox_cat.silver.orders_json
AS select from_json(fixed_value,'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') AS json_value from tv_order_fixed_int ;

In [0]:
select * from gizmobox_cat.silver.orders_json;